In [1]:
from Hamil_search import *
from simulation import *
import numpy as np
import pennylane as qml

In [2]:
def get_P0_HpenInverse(Hpen):
    """
    Make sure that the eigenvalues of Hpen are all integers
    """
    e, u = np.linalg.eigh(Hpen)

    a = np.array([1 if abs(x) < 0.5 else 0 for x in e])
    P0 = u @ (np.expand_dims(a, axis=1) * u.conj().T)

    b = np.array([1/x if abs(x) > 0.5 else 0 for x in e])
    HpenInverse = u @ (np.expand_dims(b, axis=1) * u.conj().T)

    return P0, HpenInverse

def check(HpenInverse, P0, Penc, Uenc, Henc1, Henc2):
    # check Henc2 is zero up to 1st order
    print("Check Henc2 is zero up to 1st order perturbation...")
    if checkSame(P0 @ Henc2 @ P0, np.zeros(P0.shape)):
        print("Pass")
    else:
        print("Not Pass")
        return
    
    A = Henc1 - Henc2 @ HpenInverse @ Henc2 # P0 @ A @ P0 is the effective hamiltonian up to 2nd order

    # check there is no zero-energy leakage
    print("Check there is no zero-energy leakage...")
    if checkSame((P0 - Penc) @ A @ Penc, np.zeros(P0.shape)):
        print("Pass")
    else:
        print("Not Pass")
        return

    # print logical interaction
    print("Logical interaction:")
    Hlogi = Uenc.conj().T @ A @ Uenc
    print(qml.pauli_decompose(Hlogi, hide_identity=True, pauli=True))

# 1D TFIM

In [3]:
n_block = 2
n_phys = 4 * n_block

Hpen_terms = [(1, f'X{4*i}*X{4*i+1}+3*Z{4*i}*Z{4*i+1}+X{4*i+2}*X{4*i+3}+3*Z{4*i+2}*Z{4*i+3}') 
              for i in range(n_block)]
Hpen = blocks2Mat(n_phys, Hpen_terms)
P0, HpenInverse = get_P0_HpenInverse(Hpen)
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T

Henc1_terms = [(1, f'X{4*i}*X{4*i+1}+(-1)*X{4*i}*X{4*i+2}') # logical X
               for i in range(n_block)]
Henc1_terms += [(1, f'Z{4*i}*Z{4*i+1}+Z{4*i}*Z{4*i+2}') # logical Z
               for i in range(n_block)]
Henc1_terms += [(1, f'Z{4*i+1}*Z{4*i+2}') # inner-block logical ZZ
               for i in range(n_block)]
Henc1_terms += [(1, f'Z{4*i+4}*Z{4*i+5}') # compensate for residual terms from cross-block gadget
                for i in range(n_block-1)]
Henc1 = blocks2Mat(n_phys, Henc1_terms)

Henc2_terms = [(np.sqrt(8/3), f'Z{4*i+1}*X{4*i+6}+Z{4*i+3}*X{4*i+6}')
               for i in range(n_block-1)] # cross-block logical ZZ
Henc2 = blocks2Mat(n_phys, Henc2_terms)

check(HpenInverse, P0, Penc, Uenc, Henc1, Henc2)


Check Henc2 is zero up to 1st order perturbation...
Pass
Check there is no zero-energy leakage...
Pass
Logical interaction:
0.9999999999999996 * X(3)
+ 0.9999999999999996 * Z(3)
+ 0.9999999999999997 * X(2)
+ 0.9999999999999996 * Z(2)
+ 0.9999999999999993 * Z(2) @ Z(3)
+ 0.9999999999999996 * X(1)
+ 0.9999999999999997 * Z(1)
+ 0.9999999999999994 * Z(1) @ Z(2)
+ 0.9999999999999996 * X(0)
+ 0.9999999999999997 * Z(0)
+ 0.9999999999999994 * Z(0) @ Z(1)


# 1D XZ chain

In [4]:
n_block = 2
n_phys = 4 * n_block

Hpen_terms = [(1, f'X{4*i}*X{4*i+1}+3*Z{4*i}*Z{4*i+1}+X{4*i+2}*X{4*i+3}+3*Z{4*i+2}*Z{4*i+3}') 
              for i in range(n_block)]
Hpen = blocks2Mat(n_phys, Hpen_terms)
P0, HpenInverse = get_P0_HpenInverse(Hpen)
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T

Henc1_terms = []
Henc1_terms += [(1, f'Z{4*i+1}*Z{4*i+2}+(-1)*X{4*i+1}*X{4*i+2}') # inner-block logical ZZ & XX
               for i in range(n_block)]
Henc1_terms += [(1.5, f'X{4*i}*X{4*i+1}') # compensate for residual terms from cross-block gadget
                for i in range(n_block-1)]
Henc1_terms += [(-3.5, f'Z{4*i+4}*Z{4*i+5}') # compensate for residual terms from cross-block gadget
                for i in range(n_block-1)]
Henc1 = blocks2Mat(n_phys, Henc1_terms)

Henc2_terms = [(np.sqrt(8/3), f'Z{4*i+1}*X{4*i+6}+Z{4*i+3}*X{4*i+6}+3*Z{4*i+1}*X{4*i+4}')
               for i in range(n_block-1)] # cross-block logical ZZ & XX
Henc2 = blocks2Mat(n_phys, Henc2_terms)

check(HpenInverse, P0, Penc, Uenc, Henc1, Henc2)


Check Henc2 is zero up to 1st order perturbation...
Pass
Check there is no zero-energy leakage...
Pass
Logical interaction:
0.9999999999999996 * X(2) @ X(3)
+ 0.9999999999999998 * Z(2) @ Z(3)
+ 1.0 * X(1) @ X(2)
+ 0.9999999999999996 * Z(1) @ Z(2)
+ 0.9999999999999993 * X(0) @ X(1)
+ 0.9999999999999998 * Z(0) @ Z(1)


# 2D TFIM

In [15]:
n_row = 3 # number of rows of blocks
n_col = 1 # number of columns of blocks

def idx(c, r, i):
    """
    Return the index of the physical qubit at the `i`-th position of the block located 
    at column `c` and row `r`.
    `c`, `r`, and the returned index all start from 0.
    `i` takes value from {1,2,3,4}.
    """
    assert 0 <= c and c < n_col
    assert 0 <= r and r < n_row
    assert 1 <= i and i <= 4
    return c * n_row * 4 + r * 4 + i - 1

n_block = n_row * n_col
n_phys = 4 * n_block
dim_phys = 2 ** n_phys

gx_flavors = [1, 2, 3]
gz_flavors = [4, 5, 6]

Hpen_terms = []
for c in range(n_col):
    for r in range(n_row):
        gx, gz = gx_flavors[r % 3], gz_flavors[r % 3]
        Hpen_terms += [(gx, f'X{idx(c, r, 1)}*X{idx(c, r, 2)}'), (gz, f'Z{idx(c, r, 1)}*Z{idx(c, r, 2)}'),
                       (gx, f'X{idx(c, r, 3)}*X{idx(c, r, 4)}'), (gz, f'Z{idx(c, r, 3)}*Z{idx(c, r, 4)}')]
Hpen = blocks2Mat(n_phys, Hpen_terms)
P0, HpenInverse = get_P0_HpenInverse(Hpen)
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T

Henc1_terms = []
for c in range(n_col):
    for r in range(n_row):
        Henc1_terms += [(1, f'Z{idx(c, r, 2)}*Z{idx(c, r, 3)}')]
Henc2_terms = []
for c in range(n_col):
    for r in range(n_row):
        if r < n_row - 1: # vertical cross-block interactions
            gx, gz = gx_flavors[(r+1) % 3], gz_flavors[r % 3]
            alpha = np.sqrt((gz ** 2 - gx ** 2) / gz)
            Henc2_terms += [(alpha, f'X{idx(c, r, 3)}*Z{idx(c, r+1, 2)}'), (alpha, f'X{idx(c, r, 3)}*Z{idx(c, r+1, 4)}')]
            Henc1_terms += [(1, f'Z{idx(c, r, 1)}*Z{idx(c, r, 2)}')]

            gx, gz = gx_flavors[r % 3], gz_flavors[(r+1) % 3]
            alpha = np.sqrt((gz ** 2 - gx ** 2) / gz)
            Henc2_terms += [(alpha, f'Z{idx(c, r, 2)}*X{idx(c, r+1, 3)}'), (alpha, f'Z{idx(c, r, 4)}*X{idx(c, r+1, 3)}')]
            Henc1_terms += [(1, f'Z{idx(c, r+1, 1)}*Z{idx(c, r+1, 2)}')]
        if c < n_col - 1: # horizontal cross-block interactions
            gx, gz = gx_flavors[r % 3], gz_flavors[r % 3]
            alpha = np.sqrt((gz ** 2 - gx ** 2) / gz)
            if r % 2 == 0:
                Henc2_terms += [(alpha, f'Z{idx(c, r, 2)}*X{idx(c+1, r, 3)}'), (alpha, f'Z{idx(c, r, 4)}*X{idx(c+1, r, 3)}')]
                Henc1_terms += [(1, f'Z{idx(c+1, r, 1)}*Z{idx(c+1, r, 2)}')]
            else:
                Henc2_terms += [(alpha, f'X{idx(c, r, 3)}*Z{idx(c+1, r, 2)}'), (alpha, f'X{idx(c, r, 3)}*Z{idx(c+1, r, 4)}')]
                Henc1_terms += [(1, f'Z{idx(c, r, 1)}*Z{idx(c, r, 2)}')]

Henc1 = blocks2Mat(n_phys, Henc1_terms)
Henc2 = blocks2Mat(n_phys, Henc2_terms)
check(HpenInverse, P0, Penc, Uenc, Henc1, Henc2)

Check Henc2 is zero up to 1st order perturbation...
Pass
Check there is no zero-energy leakage...
Pass
Logical interaction:
0.9999999999999993 * Z(4) @ Z(5)
+ 0.9999999999999994 * Z(3) @ Z(4)
+ 0.9999999999999997 * Z(2) @ Z(5)
+ 0.9999999999999996 * Z(2) @ Z(3)
+ 0.9999999999999991 * Z(1) @ Z(2)
+ 0.9999999999999994 * Z(0) @ Z(3)
+ 0.9999999999999993 * Z(0) @ Z(1)
